<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><span><a href="#Nodes" data-toc-modified-id="Nodes-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Nodes</a></span></li><li><span><a href="#Data-model" data-toc-modified-id="Data-model-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data model</a></span></li></ul></li><li><span><a href="#Quickstart" data-toc-modified-id="Quickstart-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Quickstart</a></span><ul class="toc-item"><li><span><a href="#Required-libraries" data-toc-modified-id="Required-libraries-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Required libraries</a></span></li><li><span><a href="#create-a-flowchart-using-the-autoplot-convenience-function" data-toc-modified-id="create-a-flowchart-using-the-autoplot-convenience-function-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>create a flowchart using the autoplot convenience function</a></span></li><li><span><a href="#Processing-and-plotting-data" data-toc-modified-id="Processing-and-plotting-data-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Processing and plotting data</a></span></li><li><span><a href="#Accessing-data-and-nodes-dynamically-with-code" data-toc-modified-id="Accessing-data-and-nodes-dynamically-with-code-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Accessing data and nodes dynamically with code</a></span></li></ul></li><li><span><a href="#Appendix:-Explicit-set-up-of-autoplot" data-toc-modified-id="Appendix:-Explicit-set-up-of-autoplot-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Appendix: Explicit set-up of autoplot</a></span><ul class="toc-item"><li><span><a href="#Setting-up-the-flowchart" data-toc-modified-id="Setting-up-the-flowchart-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Setting up the flowchart</a></span></li><li><span><a href="#Creating-the-GUI" data-toc-modified-id="Creating-the-GUI-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Creating the GUI</a></span></li><li><span><a href="#feed-in-some-data" data-toc-modified-id="feed-in-some-data-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>feed in some data</a></span></li><li><span><a href="#notebook-and-GUI-interaction" data-toc-modified-id="notebook-and-GUI-interaction-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>notebook and GUI interaction</a></span></li></ul></li></ul></div>

# Introduction

This is a basic illustration of how plottr can be used for interactive data inspection. In this notebook we go through a simple example of how plottr easily allows us to look at multidimensional data.


Let's start with some key concepts that plottr uses under the hood:


## Nodes

Data in plottr is transformed through a flowchart of nodes. This is inherited from the pyqtgraph library. Each node typically has some input data, does some processing on it, which then results in output data. This is done essentially on-demand, i.e., only when the input has changed, the node performs processing. 

## Data model

Within plotter, we use the DataDict format (see the "Working with data" example notebook). This is basically a dictionary-based data container with some standardized meta-data and some convenience functions for working on the data. Each node within plottr is expected to accept DataDicts as input and to produce DataDicts as output (with the exception of None, which signals that there is no output right now).

# Quickstart

## Required libraries

Under the hood we make heavy use of pyqt and pyqtgraph; for plotting we use matplotlib as well. Our main import block looks like this (see below for the role each of the parts play).
Note that the first code line is the `gui qt` magic, which allows us handle a Qt mainloop nicely within the notebook.

In [ ]:
%gui qt
%matplotlib qt

import logging
import numpy as np
from matplotlib import pyplot as plt

from plottr.apps import autoplot

## create a flowchart using the autoplot convenience function

To make usage easy, we can use some pre-arranged tools. A simple one is autoplot, that creates a flowchart and UI that allows the user to easily select and inspect data. 

The convenience method below will returns a pyqtgraph flowchart object, as well as the GUI window (a pyqt dialog).

In [ ]:
fc, win = autoplot.autoplot()

## Processing and plotting data

As a simple example we make some test data, and feed it into the flowchart. The data object we need to use here is of type `plottr.data.DataDict`.

**Note:** As of a recent update of pyqt the cell below might throw a Runtime exception. You can ignore this (in fact, it normally only occurs once...). If it bugs you, a simple workaround for now is just run the above cell twice, that seems to reliably circumvent the issue.

In [ ]:
import data as testdata
data = testdata.three_incompatible_3d_sets(31, 31, 21)
fc.setInput(dataIn=data)

At this point, the plot widget should display the available data in the *Data Selector* dock. The usage should be somewhat straight-forward, but in short:
* you can select compatible data sets in the selector (compatible means data with the same axes).
* We can decide how to grid the data. 
    * A good guess for now is to guess the shape based on the unique values in each dimension.
    * we could also go for no grid. in that case the data will be output as a column-based format
* the selected data is forwarded to an axes selector that allows us to make a 1d or 2d data set out of the data, which can then be plotted.
    * any axes not used as x or y is *reduced*, i.e., it's not present anymore in the output. 
    * When data is on a grid, we have some options as to how to do that. The GUI currently supports averaging the axes, or selecting a particular value of that axis (ie we slice the data).
    * if data is not a grid, we just ignore each axis that's not designated as x or y.
* finally, the selected data is plotted.
    * if data is on a grid, we use line and colormesh plots
    * if not, we use scatter plots.
    
**Note**: you can have multiple windows open. Logging might at this point behave a bit oddly if you use it, but nothing worse than that should happen.

In [ ]:
fc2, win2 = autoplot.autoplot(inputData=data)

## Accessing data and nodes dynamically with code

An important part of plottr is the integration of the GUI with interactive python use: 
* The different nodes in the flowchart can be controlled from the python interpreter
* All data is accessible interactively as well

As an example, let's use the flowchart from the autoplot app above.

In [ ]:
# print all available nodes
fc.nodes()

In [ ]:
# selecting data
fc.nodes()['DataSelector.0'].selectedData = ['data']

# automatically guess the grid shape
fc.nodes()['Gridder.0'].grid = 'guess'

# select x and y axes
fc.nodes()['XYAxesSelector.0'].xyAxes = ('x', 'y')

# ... and average over the other dimension:
# each reduction wants a function that reduces the dimension of the data by 1,
# and optional args/kwargs. Here those are simply empty.
fc.nodes()['XYAxesSelector.0'].reductions = {
    'z' : (np.mean, [], {})
}

In [ ]:
# get the data, and plot
output = fc.output()['dataOut']

xx = output.data_vals('x')
yy = output.data_vals('y')
zz = output.data_vals('data')

fig, ax = plt.subplots(1,1, dpi=150)
ax.pcolormesh(xx, yy, zz.T)
ax.set_xlabel(output.label('x'))
ax.set_ylabel(output.label('y'))

# Appendix: Explicit set-up of autoplot

NOTE: you might have to restart the kernel and run only imports and the things below. The QT libs sometimes act up when using them in 'weird' orders in the same kernel session.

In [ ]:
import pyqtgraph as pg
from pyqtgraph.flowchart import Flowchart
from pyqtgraph import Qt
from pyqtgraph.Qt import QtGui, QtCore
from pyqtgraph.flowchart import library as fclib
from pyqtgraph.dockarea import Dock, DockArea

from plottr.data.datadict import DataDict
from plottr import log as plottrlog
from plottr.node.data_selector import DataSelector
from plottr.node.grid import DataGridder
from plottr.node.dim_reducer import XYAxesSelector
from plottr.plot.mpl import PlotNode, AutoPlot

## Setting up the flowchart

We first populate the Node Library with the nodes we have at our disposal (in practice you probably want that to happen in some initialization function or script). We then instantiate the nodes we want to use, and connect the terminals in the right order. 

In [ ]:
nodelib = fclib.NodeLibrary()
nodelib.addNodeType(DataSelector, [('Basic')])
nodelib.addNodeType(DataGridder, [('Basic')])
nodelib.addNodeType(XYAxesSelector, [('Basic')])
nodelib.addNodeType(PlotNode, [('Plot')])

fc = Flowchart(terminals={
    'dataIn': {'io': 'in'},
    'dataOut': {'io': 'out'}
})
fc.library = nodelib

datasel = fc.createNode('DataSelector')
grid = fc.createNode('Gridder')
xysel = fc.createNode('XYAxesSelector')
plot = fc.createNode('Plot')

fc.connectTerminals(fc['dataIn'], datasel['dataIn'])
fc.connectTerminals(datasel['dataOut'], grid['dataIn'])
fc.connectTerminals(grid['dataOut'], xysel['dataIn'])
fc.connectTerminals(xysel['dataOut'], fc['dataOut'])
fc.connectTerminals(xysel['dataOut'], plot['dataIn'])

## Creating the GUI

This is basically now just arranging the different widgets in a reasonable way. All the nodes we already instantiated provide ui widgest per default (not generally true, but for these Nodes it is :)). 

To avoid confusion when handling messages and user information from the flowchart and nodes we use a custom log handler. All log messages sent to a logger with name "plottr.\*" will be captured.

Since plottr is GUI tool, it is only fitting to use a GUI output for all logs :)
Here we open a separate dialog which contains text output from plottr.

In [ ]:
# Setting up the GUI window
area = DockArea()
layout = QtGui.QVBoxLayout()
layout.addWidget(area)
win = QtGui.QDialog()
win.setLayout(layout)

# data selector
dataselDock = Dock('Data Selector', size=(150, 100))
dataselDock.addWidget(datasel.ui)
area.addDock(dataselDock)

# grid
gridDock = Dock('Grid', size=(100, 100))
gridDock.addWidget(grid.ui)
area.addDock(gridDock, 'right', dataselDock)

# xy selector
xyselDock = Dock('XY Axes Selector', size=(250, 100))
xyselDock.addWidget(xysel.ui)
area.addDock(xyselDock, 'bottom')

# log
logDock = Dock('Log', size=(250, 100))
logDock.addWidget(plottrlog.setupLogging(makeDialog=False))
area.addDock(logDock, 'bottom', xyselDock)

# plot widget
plotWidget = AutoPlot()
plot.setPlotWidget(plotWidget)
plotDock = Dock('Plot', size=(500, 300))
plotDock.addWidget(plotWidget)
area.addDock(plotDock, 'right')

win.show()

### Logging
logger = plottrlog.getLogger()
plottrlog.LEVEL = logging.DEBUG

## feed in some data

After plugging in some input data we can start exploring it using the GUI.

In [ ]:
import data
testdata = data.three_incompatible_3d_sets(101, 101, 201)
fc.setInput(dataIn=testdata)

## notebook and GUI interaction

As already seen above, we can change the options also using code in the notebook, and we can get the data from the flowchart output and process it further.

In [ ]:
datasel.selectedData = ['data']
grid.grid = 'guess'
xysel.xyAxes = 'x', 'y'
xysel.reductions = {
    'z' : (np.mean, [], {})
}